In [1]:
import sys
import os
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from copy import deepcopy
import joblib
from statannot import add_stat_annotation
import xgboost as xgb

# sklearn
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, SelectFromModel, SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from  sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, IsolationForest
from sklearn.linear_model import Lasso, LassoCV, RidgeCV, LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# imbalance learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# deepnet 
sys.path.insert(0, 'ENNS')
from dnp2 import DeepNet

import warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

# Read features and HR phenotypes

In [2]:
df_X = pd.read_csv('feature_table_combined.csv', index_col=0)
df_y = pd.read_csv("hr_phenotype.csv", index_col=0)[['HR']].astype(int).loc[df_X.index]

In [3]:
df_X.head()

,CNV_301,CNV_302,CNV_3,CNV_2,CNV_124,CNV_16,CNV_123,CNV_1,CNV_168,CNV_13,...,VAR_3881,VAR_3882,VAR_3883,VAR_4854,VAR_4855,VAR_4850,VAR_4851,VAR_4852,VAR_1207,VAR_4853
MSK1004,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,2,2
MSK1015,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,2,2
MSK1082,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
MSK1090,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
MSK1119,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,2,2


In [4]:
df_y.head()

,HR
MSK1004,1
MSK1015,1
MSK1082,0
MSK1090,0
MSK1119,1


In [21]:
dict_var_map = joblib.load('variant_id_mapping.joblib')
dict(list(dict_var_map.items())[0:5])

{'cpar_Chr_1:1000485:SNP:A:G': 'VAR_448',
 'cpar_Chr_1:1001145:SNP:A:G': 'VAR_16',
 'cpar_Chr_1:1001160:SNP:C:T': 'VAR_35',
 'cpar_Chr_1:1001800:SNP:T:A': 'VAR_115',
 'cpar_Chr_1:1002115:SNP:T:C': 'VAR_7'}

In [22]:
dict_cnv_map = joblib.load('copynumber_id_mapping.joblib')
dict(list(dict_cnv_map.items())[0:5])

{'CPAR2_100010': 'CNV_301',
 'CPAR2_100020': 'CNV_302',
 'CPAR2_100030': 'CNV_3',
 'CPAR2_100040': 'CNV_3',
 'CPAR2_100050': 'CNV_2'}

# Remove outliers using IsolationForest

In [5]:
iso = IsolationForest(contamination='auto', n_estimators=1000, random_state=42)
yhat = iso.fit_predict(df_X) # abnormal samples will be labeled -1 in yhat
mask = yhat != -1 # mask is a boolean array that indicates the samples to keep
print('outliers that will be removed from the dataset:', (',').join(list(df_X.index[yhat==-1])))
df_X = df_X.loc[mask]
df_y = df_y.loc[mask]

outliers that will be removed from the dataset: UWM1206,MSK520,MSK846,CDC339,FM14


# Remove redundant isoaltes: pick one isolate among all identical ones within each patient

In [6]:
df_cluster = pd.read_csv("strain_clusters_popANI_99_999.csv", index_col=0)
df_cluster = df_cluster.loc[df_X.index]
df_cluster.head()

,ClusterName,Institute,Location,HR,Weight,PatientID
MSK1004,0,MSK,USA,True,0.000322,MSK_Pt_11
MSK1015,0,MSK,USA,True,0.000322,MSK_Pt_11
MSK1082,34,MSK,USA,False,0.001330,MSK_Pt_26
MSK1090,34,MSK,USA,False,0.001330,MSK_Pt_26
MSK1119,0,MSK,USA,True,0.000322,MSK_Pt_3


In [7]:
print("%d out of %d isolates have patient information."%(len(df_cluster[df_cluster.PatientID.notnull()]), len(df_cluster)))

103 out of 228 isolates have patient information.


In [9]:
df_ani = pd.read_csv("popANI_pairwise.csv", index_col=0)
df_ani = df_ani.loc[df_X.index, df_X.index]
df_ani.head()

,MSK1004,MSK1015,MSK1082,MSK1090,MSK1119,MSK1129,MSK1191,UWM1195,UWM1196,UWM1197,...,MSK314,MSK315,MSK33,MSK34,MSK478,MSK49,MSK5,MSK51,MSK65,MSK67
MSK1004,1.000000,1.000000,0.999758,0.999757,0.999997,0.999997,0.999937,0.999752,0.999751,0.999777,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
MSK1015,1.000000,1.000000,0.999757,0.999756,0.999997,0.999997,0.999937,0.999750,0.999749,0.999775,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
MSK1082,0.999758,0.999757,1.000000,1.000000,0.999753,0.999756,0.999780,0.999963,0.999957,0.999799,...,0.999758,0.999758,0.999761,0.999760,0.999759,0.999758,0.999756,0.999759,0.999759,0.999756
MSK1090,0.999757,0.999756,1.000000,1.000000,0.999752,0.999755,0.999779,0.999963,0.999957,0.999798,...,0.999757,0.999757,0.999760,0.999759,0.999758,0.999757,0.999756,0.999757,0.999758,0.999755
MSK1119,0.999997,0.999997,0.999753,0.999752,1.000000,1.000000,0.999936,0.999748,0.999746,0.999771,...,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997,0.999997


In [10]:
selected_isolates = []
for pid in set(df_cluster[df_cluster.PatientID.notnull()].PatientID):
    df_pid = df_cluster[df_cluster.PatientID==pid] # select isolates from the patient pid
    for cluster in set(df_pid.ClusterName):
        df_pid_cluster = df_pid[df_pid.ClusterName==cluster] # select isolates from each cluster within the patient

        # identify the mean distance between this selected isolate and every other isolates
        df_mean_ani = df_ani.mean(axis=1)[list(df_pid_cluster.index)]

        # select the isolate with longest distance to all other isolates
        isolate_most_dissimilar = df_mean_ani.sort_values().index[0]

        # output information
        print('Patient ID: %s, Cluster: %s'%(str(pid), str(cluster)))
        print('This patient and cluster combination has %d isolates, %d HR and %d S'%(
            len(df_pid_cluster),
            len(df_pid_cluster[(df_pid_cluster.PatientID==pid) & (df_pid_cluster.HR==True)]),
            len(df_pid_cluster[(df_pid_cluster.PatientID==pid) & (df_pid_cluster.HR==False)])
        ))
        print(isolate_most_dissimilar, '(HR=%d) was selected'%(df_pid_cluster.loc[isolate_most_dissimilar,'HR']))
        print()

        selected_isolates.append(isolate_most_dissimilar)
    
df_X = df_X.loc[list(df_cluster[df_cluster.PatientID.isnull()].index)+selected_isolates]
df_y = df_y.loc[df_X.index]

Patient ID: MSK_Pt_29, Cluster: 1
This patient and cluster combination has 2 isolates, 1 HR and 1 S
MSK2524 (HR=0) was selected

Patient ID: MSK_Pt_11, Cluster: 0
This patient and cluster combination has 2 isolates, 2 HR and 0 S
MSK1015 (HR=1) was selected

Patient ID: MSK_Pt_11, Cluster: 72
This patient and cluster combination has 1 isolates, 0 HR and 1 S
MSK1666 (HR=0) was selected

Patient ID: MSK_Pt_30, Cluster: 0
This patient and cluster combination has 7 isolates, 7 HR and 0 S
MSK314 (HR=1) was selected

Patient ID: MSK_Pt_30, Cluster: 35
This patient and cluster combination has 4 isolates, 0 HR and 4 S
MSK250 (HR=0) was selected

Patient ID: MSK_Pt_30, Cluster: 77
This patient and cluster combination has 1 isolates, 0 HR and 1 S
MSK247 (HR=0) was selected

Patient ID: PUMCH_Pt_4, Cluster: 29
This patient and cluster combination has 2 isolates, 0 HR and 2 S
E59 (HR=0) was selected

Patient ID: MSK_Pt_10, Cluster: 24
This patient and cluster combination has 2 isolates, 0 HR and 2 

In [11]:
Counter(df_cluster.loc[df_X.index].HR)

Counter({False: 113, True: 57})

# Test Model Performance in train(80%)-test(20%) split over 100 runs

## Here we try two feature selection methods (Ftest, DNP), three models (Logistic regression, RF, XGBoost), four rebalance methods (no rebalance, random oversampling, SMOTE, ADASYN), 10 numbers of features (1-10)

In [13]:
results = []
for i in [0]:#np.arange(0,100):

    # train-test split
    # the same random_state will produce the same split (used for reproducibility)
    X_train0, X_test, y_train0, y_test = train_test_split(df_X, df_y.values.ravel(), test_size=0.2, random_state=i)
    
    for RSmethod in ['NoRS','RandomOS','SMOTE','ADASYN']:
        if RSmethod=='RandomOS':
            X_train, y_train = RandomOverSampler(random_state=42).fit_resample(deepcopy(X_train0), deepcopy(y_train0))
        elif RSmethod=='SMOTE':
            X_train, y_train = SMOTE(random_state=42).fit_resample(deepcopy(X_train0), deepcopy(y_train0))
        elif RSmethod=='ADASYN':
            X_train, y_train = ADASYN(random_state=42).fit_resample(deepcopy(X_train0), deepcopy(y_train0))
        elif RSmethod=='NoRS':
            X_train = deepcopy(X_train0)
            y_train = deepcopy(y_train0)
    
        # identify numerical features and categorical features
        numeric_features = [col for col in X_train.columns if col.startswith('CNV')]
        categorical_features = [col for col in X_train.columns if col.startswith('VAR')]

        #------------------------------
        # construct pipeline components

        # preprocessing
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat',OneHotEncoder(handle_unknown="ignore"),categorical_features),
                ('num',StandardScaler(),numeric_features)
            ]
        )

        # variance threshold
        variance_thresholder = VarianceThreshold()

        # feature selection
        feature_selector_ftest = SelectKBest(score_func=f_classif, k=10) # F-score
        class feature_selector_dnp(BaseEstimator, TransformerMixin):
            def __init__(self, n_features=10):
                self.n_features = n_features
                self.selected_column_indice = None
            def fit(self, X, y=None):
                slc = DeepNet(max_feature=self.n_features)
                self.selected_column_indice = [idx-1 for idx in slc.train(X, y, return_select=True, verbosity=2)]
                return self
            def transform(self, X, y=None, **kwargs):
                return X[:, self.selected_column_indice]
        feature_selectors = {'Ftest':feature_selector_ftest,
                             'DNP':feature_selector_dnp()}
        
        # classifier
        xgboost = xgb.XGBClassifier(n_estimators=1000, seed=42, objective='binary:logistic')
        random_forest = RandomForestClassifier(n_estimators=1000, random_state=42)
        logistic_regression = LogisticRegression(random_state=42, solver='saga')
        classifiers = {'Logit':logistic_regression, 'RF':random_forest, 'XGB':xgboost}

        # classification
        for FSmethod in ['Ftest','DNP']:
            for Model in ['Logit','RF','XGB']:
                for nf in [1]:
                #for nf in np.arange(1,11): # number of features

                    # set up tuning parameters
                    if Model=='XGB':
                        if FSmethod == 'Ftest':
                            parameters = {
                                'feature_selector__k':[nf],
                                'classifier__colsample_bytree':[0.5,0.75,1.0],
                                'classifier__max_depth':[2,6,12],
                                'classifier__min_child_weight':[1,5,15],
                                'classifier__learning_rate':[0.3,0.1,0.03]
                            }
                        elif FSmethod=='DNP':
                            parameters = {
                                'feature_selector__n_features':[nf],
                                'classifier__colsample_bytree':[0.5,0.75,1.0],
                                'classifier__max_depth':[2,6,12],
                                'classifier__min_child_weight':[1,5,15],
                                'classifier__learning_rate':[0.3,0.1,0.03]
                            }
                    elif Model=='RF':
                        if FSmethod == 'Ftest':
                            parameters = {
                                'feature_selector__k':[nf],
                                'classifier__max_features':[None,'sqrt','log2'],
                                'classifier__max_depth':[4,8,16],
                                'classifier__min_samples_split':[2,4,8],
                                'classifier__min_samples_leaf':[1,2,4]
                            }
                        elif FSmethod=='DNP':
                            parameters = {
                                'feature_selector__n_features':[nf],
                                'classifier__max_features':[None,'sqrt','log2'],
                                'classifier__max_depth':[4,8,16],
                                'classifier__min_samples_split':[2,4,8],
                                'classifier__min_samples_leaf':[1,2,4]
                            }
                    elif Model=='Logit':
                        if FSmethod == 'Ftest':
                            parameters = {
                                'feature_selector__k':[nf],
                                'classifier__C':np.logspace(-3,3,7),
                                'classifier__penalty':["l1","l2"]
                            }
                        elif FSmethod=='DNP':
                            parameters = {
                                'feature_selector__n_features':[nf],
                                'classifier__C':np.logspace(-3,3,7),
                                'classifier__penalty':["l1","l2"]
                            }

                    # create pipeline
                    pipeline = Pipeline(
                        steps=[
                            ("preprocessor", preprocessor), 
                            ("thresholder", variance_thresholder),
                            ("feature_selector", feature_selectors[FSmethod]), 
                            ("classifier", classifiers[Model])]
                    )

                    # run grid search
                    CV = GridSearchCV(pipeline, parameters, scoring='f1', cv=5, n_jobs=-1, verbose=1)
                    CV.fit(X_train,y_train)

                    # compute test scores
                    y_pred = CV.predict(X_test)
                    precision = precision_score(y_true=y_test, y_pred=y_pred)
                    recall = recall_score(y_true=y_test, y_pred=y_pred)
                    f1 = f1_score(y_true=y_test, y_pred=y_pred)

                    # save to results
                    out = [i, RSmethod, FSmethod, Model, nf, 
                           (',').join(X_test.index), (',').join([str(y1) for y1 in y_test]), (',').join([str(y1) for y1 in y_pred]),
                           precision, recall, f1]
                    if Model=='XGB':
                        if FSmethod == 'Ftest':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__colsample_bytree','classifier__max_depth','classifier__min_child_weight','classifier__learning_rate','feature_selector__k']]))
                        elif FSmethod == 'DNP':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__colsample_bytree','classifier__max_depth','classifier__min_child_weight','classifier__learning_rate','feature_selector__n_features']]))
                    elif Model=='RF':
                        if FSmethod == 'Ftest':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__max_depth','classifier__max_features','classifier__min_samples_leaf','classifier__min_samples_split','feature_selector__k']]))
                        elif FSmethod == 'DNP':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__max_depth','classifier__max_features','classifier__min_samples_leaf','classifier__min_samples_split','feature_selector__n_features']]))
                    elif Model=='Logit':
                        if FSmethod == 'Ftest':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__C','classifier__penalty','feature_selector__k']]))
                        elif FSmethod == 'DNP':
                            out.append((';').join([v.split('__')[1]+':'+str(CV.best_params_[v]) 
                                                   for v in ['classifier__C','classifier__penalty','feature_selector__n_features']]))

                    results.append(out)

df_results = pd.DataFrame(results, columns = ['RandomNumber', 'RSMethod', 'FeatureSelectionMethod','Model','NumFeatures',
                                              'TestSetIsolates', 'TestSetObservedHR', 'TestSetPredictedHR',
                                              'Precision','Recall','F1','BestParams'])
df_results.to_csv("hr_classification_repeat1.final.csv", index=False)
df_results.head()

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Fitting 5 folds for each of 14 candidates, totalling 70 fits
Fitting 5 fold

,RandomNumber,RSMethod,FeatureSelectionMethod,Model,NumFeatures,TestSetIsolates,TestSetObservedHR,TestSetPredictedHR,Accuracy,Recall,F1,BestParams
0,0,NoRS,Ftest,Logit,1,"MSK2499,MSK2406,MSK2161,E31,E33,MSK1258,UWM120...","0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,...","0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...",1.0,0.545455,0.705882,C:0.1;penalty:l1;k:1
1,0,NoRS,Ftest,RF,1,"MSK2499,MSK2406,MSK2161,E31,E33,MSK1258,UWM120...","0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,...","0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...",1.0,0.545455,0.705882,max_depth:4;max_features:None;min_samples_leaf...
2,0,NoRS,Ftest,XGB,1,"MSK2499,MSK2406,MSK2161,E31,E33,MSK1258,UWM120...","0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,...","0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...",1.0,0.545455,0.705882,colsample_bytree:0.5;max_depth:2;min_child_wei...
3,0,NoRS,DNP,Logit,1,"MSK2499,MSK2406,MSK2161,E31,E33,MSK1258,UWM120...","0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,...","0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...",1.0,0.545455,0.705882,C:0.1;penalty:l1;n_features:1
4,0,NoRS,DNP,RF,1,"MSK2499,MSK2406,MSK2161,E31,E33,MSK1258,UWM120...","0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,1,1,...","0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,...",1.0,0.545455,0.705882,max_depth:4;max_features:None;min_samples_leaf...
